## Interactive Spectral Energy distribution

The goal of this tool is to show you how physical parameters impact the observed multi-wavelength flux.<br/>
The module used here is naima (http://naima.readthedocs.io/en/latest/) a package for computation of non-thermal radiation from relativistic particle populations and Markov Chain Monte Carlo fitting.  <br/>

To install naima:

conda config --add channels astropy --add channels sherpa #to add astropy affiliated packages to conda manager <br/>
conda install naima



In [1]:
%matplotlib inline

from naima import models
import naima
from astropy import units as u
from astropy import table
from astropy.io import ascii

from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
from ipywidgets.widgets.interaction import interact

def plotSED_data(SEDfile,label='data',color='blue'):
    print('Reading file =', SEDfile)
    SED = table.Table.read( SEDfile, hdu=1 )        
    plt.errorbar(E[index],E2flux[index],yerr=arrowfraction*E2flux[index],uplims=True,fmt="none",ecolor=color)

def plotSED_sync_ic( dist= 1 *u.kpc, We=1e47 *u.erg, Eemin=1*u.MeV,
                    e_cut = 100*u.TeV,B=10 *u.uG,field="CMB", rho=1*u.cm**-3,index1=2,N=200):
    """
    Distance needs to be in kpc
    """
    Eunits = u.eV
    Elow = np.logspace(-7,10, N) * Eunits # photon energies
    Ehigh = np.logspace(0,15, N) * Eunits # photon energies

    electrondist = models.ExponentialCutoffPowerLaw(
        amplitude=u.Quantity(1e36,"1/eV"), e_0= 1*Eunits,alpha=index1, e_cutoff=e_cut, beta=2)

    ic = models.InverseCompton( electrondist, seed_photon_fields=field, Eemin=Eemin )
    syn = models.Synchrotron(electrondist, B=B,Eemin=Eemin)
    brem = models.Bremsstrahlung(electrondist,n0=rho,Eemin=Eemin)
    
    ic.set_We(We, Eemin=Eemin, Eemax=100 *u.TeV)
    syn.set_We(We, Eemin=Eemin, Eemax=100 *u.TeV)
    brem.set_We(We, Eemin=Eemin, Eemax=100 *u.TeV)
    plt.loglog(Elow, syn.sed(Elow,distance=dist ),'-', alpha=0.8,label='Synchrotron')
    plt.loglog(Ehigh, ic.sed(Ehigh,distance=dist ),'-', alpha=0.8,label='Inverse Compton')
    plt.loglog(Ehigh, brem.sed(Ehigh,distance=dist ),'-', alpha=0.8,label='Bremsstrahlung')

    
#    for f in field:
#        ic = models.InverseCompton( electrondist, seed_photon_fields=[f],Eemin=Eemin )
#        ic.set_We(We, Eemin=Eemin, Eemax=100 *u.TeV)
#        plt.loglog(Ehigh, ic.sed(Ehigh,distance=dist ),'-', alpha=0.8,label=f'IC {f[0]}')
        

def plotSED_pi0( dist = 1*u.kpc, rho=1* u.cm**-3, B=10 *u.uG,
                Wp=1e50 *u.erg, e_cut = 100*u.TeV,  index=2,N=200):
    """
    Distance needs to be in kpc and density in cm-3
    """
    
    Eunits = u.eV
    E = np.logspace(-8,15.5, N) * Eunits # photon energies
    partdist = models.ExponentialCutoffPowerLaw(
        amplitude=u.Quantity(1e36,"1/eV"), e_0= 1*Eunits, alpha=index, e_cutoff=e_cut)
    
    pi0 = models.PionDecay(partdist, nh = rho)
#    psynch = models.ProtonSynchrotron(partdist, B=B)
    
    pi0.set_Wp(Wp, Epmin=1 *u.GeV, Epmax=300 *u.TeV)
#    psynch.set_Wp(Wp, Epmin=1 * u.GeV, Epmax=300 *u.TeV)
    
    plt.loglog(E, pi0.sed(E,distance=dist ),'-', alpha=0.8,label='$\pi^{\circ}$ decay')
#    plt.loglog(E, psynch.sed(E,distance=distance )*1e2,'-', alpha=0.8,label='proton Synch')

    
def plot_sens(ax, myfile,ls='--',label=''):
    tab = ascii.read(myfile)
    ax.plot(tab['energy'], tab['sensitivity'],ls=ls,label=label)

In [2]:

#dist = 5 * u.kpc
rho = 1.0 * u.cm**-3
We = 1e47 *u.erg
Wp = 1e50 *u.erg
Ecut_p = 100 *u.TeV
Eemin = 1 *u.MeV
Ecut_e = 100 *u.TeV
tel_dict={"Radio":[4e-7,4e-3],"X-ray":[0.5e3,100e3],
          "Fermi-LAT":[1e8,5e11],"Cherenkov":[1e11,300e12]}


In [3]:
def plotSED(dist=1,rho=1,Ecut_p=Ecut_p,Eemin=Eemin,Ecut_e=Ecut_e,index1=2,B=50,We=47,save=False):
    """
    Plot SED from an electron+proton population with Powerlaw + Exp cutoff distribution.
    Parameters:
    Index     :   Slope of the particle population
    Ecut_e/p  : Cutoff energy in TeV
    rho       : for the Pi0 decay, density of medium colliding with accelerated protons in cm-3
    B         : Magnetic field (microGauss) for synchrotron emission
    
    Notes:
    For the IC mechanism, only the CMB is assumed but naima supports other radiation field.
    """
    
    fig = plt.figure(figsize=(12,6))
    
    FIR_temp = 26 *u.K
    Uph = 1 *u.eV/u.cm**3
    print(dist)
    plotSED_sync_ic(dist=dist*u.kpc,We=(10**We)*u.erg, index1=index1,field = ['CMB', ['FIR', FIR_temp, Uph]],
                    Eemin=Eemin*u.MeV,e_cut=Ecut_e*u.TeV, B=B*u.uG,rho=rho*u.cm**-3)
    plotSED_pi0(dist=dist*u.kpc,rho=rho*u.cm**-3,Wp=1e50*u.erg, e_cut=Ecut_p*u.TeV, B=B*u.uG,  index=index1)
#    plotSED_data()

#    data=get_data_rxj('fullSNR')
#    data=get_data_casA('SE_filament')
#    datax=get_data_CasA_xray('allSNR')
#    naima.plot_data(datax,figure=fig)
#    datar=get_data_CasA_radio()
#    naima.plot_data(datar,figure=fig)

    
    
    ymax=1e-8
    plt.ylim(3e-16,ymax)
    plt.xlim(3e-7,1e15)
    size=14
    plt.ylabel(r'$E^2$ d$N$/d$E$ (erg cm$^{-2}$ s$^{-1}$)',fontsize=size)
    plt.xlabel(r'$E$ (eV)',fontsize=size)
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.e') )
    for tel in tel_dict:
        ax.axvspan(tel_dict[tel][0],tel_dict[tel][1],alpha=0.2,color='grey')
        plt.text(tel_dict[tel][0],ymax*1.13,tel, fontsize=size)

    
    plot_sens(ax, 'LHAASO-sensitivity-1yr.dat', ls='--',label='LHAASO 1yr')    
    plot_sens(ax, 'LHAASO-sensitivity-5yr.dat', ls='-',label='LHAASO 5yrs')
    
        
    plt.legend(numpoints=1, loc='upper left')
    if save :
        outfile=f'SNR_d{dist}kpc_B{B}_rho{rho}_index{index1}_Ece{Ecut_e}_Ecp{Ecut_p}_We{We}.pdf'
        plt.savefig(outfile,dpi=100)
        print('File saved to : %s'%(outfile))
    plt.show()


interact(plotSED,dist=[1,3,5,8],Ecut_p=(1e1,1.01e3,1e2),Eemin=(1,100,2.5),Ecut_e=(1e1,50,2.5),index1=(1.75,2.5,0.1),
         rho=(0.1,30,1), B=(10,100,5),We=(46.5,49.5,0.1),save=False )

interactive(children=(Dropdown(description='dist', options=(1, 3, 5, 8), value=1), FloatSlider(value=1.0, desc…

<function __main__.plotSED(dist=1, rho=1, Ecut_p=<Quantity 100. TeV>, Eemin=<Quantity 1. MeV>, Ecut_e=<Quantity 100. TeV>, index1=2, B=50, We=47, save=False)>